# Decision Tree implementation OR table

In [75]:
import numpy as np
import pandas as pd

data = np.array([
    [0,1,1],
    [1,1,1],
    [1,0,1],
    [0,0,0]
])

data = pd.DataFrame(data)
data.columns= ['0','1','Y']
data

,0,1,Y
0,0,1,1
1,1,1,1
2,1,0,1
3,0,0,0


In [79]:
# Defining entropy 

def entropy(data):
    ent = 0.0
    N = float(len(data))
    counts = np.unique(data, return_counts = True)
    
    for i in counts[1]:
        p = i/N
        ent += -1.0*p*np.log2(p)
    
    return ent

In [87]:
# this function will be used for dividing data into two parts values greater than mean in one part and values less than mean on the other part.

def divide_data(data, feature, val):
    x_right = pd.DataFrame([], columns = data.columns)
    x_left = pd.DataFrame([], columns = data.columns)
    
    for i in range(data.shape[0]):
        val_ = data[feature].loc[i]
        
        if val_ > val:
            x_right.loc[len(x_right.index)] = data.loc[i].values
        else:
            x_left.loc[len(x_left.index)] = data.loc[i].values
            
    return x_left, x_right
        

In [88]:
# Defining information gain

def information_gain(data, feature, val):
    left, right = divide_data(data, feature, val)
    
    l = float(left.shape[0] / data.shape[0])
    r = float(right.shape[0] / data.shape[0])
        
    if left.shape[0] == 0 or right.shape[0] == 0 :
        return -1000000
    
    i_gain = entropy(data) - (l*entropy(left) +  r*entropy(right))
    
    return i_gain

In [89]:
# maintain the count of 0 and 1

def find_count(data):
    count = []
    count.append(data[data['Y']==0].shape[0])
    count.append(data[data['Y']==1].shape[0])

    

    return count


In [98]:
class DecisionTree:
    #Constructor
    def __init__(self,depth=0, max_depth = 2):
        self.left = None
        self.right = None
        self.feature = None
        self.val = None
        self.count = None
        self.depth = depth
        self.target = None
        self.max_depth = max_depth
        
    def fit(self, data):
        output = [0,1]
        features = ['0','1']
        info_gains = []
        
        for i in features:
            i_gain = information_gain(data, i, data[i].mean())
            info_gains.append(i_gain)
        
        self.feature = features[np.argmax(info_gains)]
        self.val = data[self.feature].mean()
        print('Level ', self.depth)
        self.count = find_count(data)
        cnt = 0
        
        for i in range(len(self.count)):
            if self.count[i]:
                print('Count of ', output[i], ' = ', self.count[i])
                cnt += 1
        print('Current Entropy = ', entropy(data.Y))
        if cnt != 1:
            print('Splitting on Tree Features ', self.feature, 'with information gain ',max(info_gains))
            
        # split data
        data_left, data_right = divide_data(data, self.feature, self.val)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop= True)
        
        
        print(cnt)
        
        # Truly a left node
        if cnt == 1:
            if data.Y.mean() >= 0.5:
                self.target = output[1]
            else:
                self.target = output[0]
            print('Reached Leaf Node')
            print()
            print()
            
        # Stop early
        if (self.depth >= self.max_depth):
            if data.Y.mean() >= 0.5:
                self.target = output[1]
            else:
                self.target = output[0]
            print('Max depth Reached')
            print()
            print()
            return
        
        print()
        print()
        
        
        # Recursive case
        self.left = DecisionTree(depth = self.depth+1, max_depth = self.max_depth)
        self.left.fit(data_left)
        
        self.right = DecisionTree(depth = self.depth+1, max_depth = self.max_depth)
        self.right.fit(data_right)
        
        if data.Y.mean() >= 0.5:
            self.target = output[1]
        else:
            self.target = output[0]
            
        return
    

In [99]:
clf = DecisionTree()
clf.fit(data)

Level  0
Count of  0  =  1
Count of  1  =  3
Current Entropy =  0.8112781244591328
Splitting on Tree Features  0 with information gain  0.587128886399193
2


Level  1
Count of  0  =  1
Count of  1  =  1
Current Entropy =  1.0
Splitting on Tree Features  1 with information gain  1.3774437510817341
2


Level  2
Count of  0  =  1
Current Entropy =  0.0
1
Reached Leaf Node


Max depth Reached


Level  2
Count of  1  =  1
Current Entropy =  0.0
1
Reached Leaf Node


Max depth Reached


Level  1
Count of  1  =  2
Current Entropy =  0.0
1
Reached Leaf Node




Level  2
Count of  1  =  1
Current Entropy =  0.0
1
Reached Leaf Node


Max depth Reached


Level  2
Count of  1  =  1
Current Entropy =  0.0
1
Reached Leaf Node


Max depth Reached


